# 1. Import of data from CSV file

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

data2=pd.read_csv('fresh.csv')
data3=pd.read_csv('sintered.csv')

# 2. Plotting the activity

In [ ]:
from scipy.interpolate import make_interp_spline

# Fresh sample data
T1 = data2.loc[[3,7,9,10], 'Temp (°C)']
C_in1 = data2.loc[[3,7,9,10], '[NO] (ppm)']
C_out1 = data2.loc[[3,7,9,10], '[NO]ₒᵤₜ (ppm)']

# Sintered sample data
T2 = data3.loc[[0,1,4,5], 'Temp (°C)']
C_in2 = data3.loc[[0,1,4,5], '[NO] (ppm)']
C_out2 = data3.loc[[0,1,4,5], '[NO]ₒᵤₜ (ppm)']


def conversion(C_in, C_out):
    C_in_np = np.array(C_in)
    C_out_np = np.array(C_out)
    return (1 - (C_out_np / C_in_np)) * 100

# Calculate conversions
X1 = conversion(C_in1, C_out1)  # Fresh sample
X2 = conversion(C_in2, C_out2)  # Sintered sample

# Smooth curves (interpolation)
def smooth_curve(T, X):
    T_sorted = np.sort(T)
    X_sorted = X[np.argsort(T)]
    spline = make_interp_spline(T_sorted, X_sorted) #fitting a smooth curve that passes through all the sorted data points.
    T_smooth = np.linspace(min(T), max(T), 300)
    X_smooth = spline(T_smooth)
    return T_smooth, X_smooth

T_smooth1, X_smooth1 = smooth_curve(T1, X1)  # Fresh
T_smooth2, X_smooth2 = smooth_curve(T2, X2)  # Sintered

plt.figure(figsize=(8, 5))
plt.plot(T_smooth1, X_smooth1, 'b-', label='Fresh Sample')
plt.scatter(T1, X1, color='blue')
plt.plot(T_smooth2, X_smooth2, 'g-', label='Sintered Sample')
plt.scatter(T2, X2, color='green' )

plt.xlabel('Temperature (°C)', fontsize=14, weight= 'bold')
plt.ylabel('% NO Conversion', fontsize=14,weight= 'bold')
plt.title('Activity of Fresh vs. Sintered Samples', fontsize=14, weight= 'bold')
plt.grid(True, linestyle='-')
plt.legend(fontsize=10)
plt.xlim(270,330)
plt.ylim(0, 20)
plt.show()

# 3. calculation of activation energy from Arrhenius plot

In [ ]:
# To get Arrhenius plot and estimate activation energy, we need to use data points where gas composition is constant but temperature is changing
# and conversion is less than 10%.
# For fresh sample, 4 data points are available for Arrhenius plot.
rate_constant = data2.loc[[8,9,10,11],'Rate (µmol/g·s)'] #importing rate constant from CSV file of fresh catalyst sample
T = data2.loc[[8,9,10,11],'Temp (°C)'] # importing Temperature
lnk = np.log(rate_constant) #finding logarithm of rate constant
inv_T = 1/T

inv_T_np = inv_T.to_numpy().reshape(-1, 1) #converting to numpy from pandas
lnk_np = lnk.to_numpy()

lin_model = LinearRegression().fit(inv_T_np, lnk_np) #fitting a linear model to estimate activation energy.
r_squared = lin_model.score(inv_T_np, lnk_np) #R^2 value
rmse = np.sqrt(mean_squared_error(lnk_np, lin_model.predict(inv_T_np))) #RMSE value
Ea = -lin_model.coef_[0] * 8.314  # Slope of the straight line is activation energy (J/mol) for fresh sample.

print(f"Activation energy of fresh catalyst : {Ea:.0f} J/mol")
print(f"R2 value: {r_squared:.2f}")
print(f"RMSE value: {rmse:.2f}")

plt.figure(figsize=(7,5))
plt.scatter(inv_T, lnk, color='Blue', label="Original Data")
plt.plot(inv_T, lin_model.predict(inv_T.to_numpy().reshape(-1,1)), label="Linear Fit", color='Magenta')
plt.xlabel("1/T (1/K)")
plt.ylabel("ln(k)")
plt.title("Arrhenius Plot of Fresh sample")
plt.legend()
plt.grid()
plt.show()

# For sintered sample, 4 data points are available for Arrhenius plot.
rate_constant = data3.loc[[0,1,4,5],'Rate (µmol/g·s)'] #importing rate constant from CSV file of sintered catalyst sample
T = data3.loc[[0,1,4,5],'Temp (°C)'] # importing Temperature
lnk = np.log(rate_constant) #finding logarithm of rate constant
inv_T = 1/T

inv_T_np = inv_T.to_numpy().reshape(-1, 1) #converting to numpy from pandas
lnk_np = lnk.to_numpy()

lin_model = LinearRegression().fit(inv_T_np, lnk_np) #fitting a linear model to estimate activation energy.
r_squared = lin_model.score(inv_T_np, lnk_np) #R^2 value
rmse = np.sqrt(mean_squared_error(lnk_np, lin_model.predict(inv_T_np))) #RMSE value
Ea = -lin_model.coef_[0] * 8.314 # Slope of the straight line is activation energy (J/mol) for sintered sample.

print(f"")
print(f"Activation energy of sintered catalyst : {Ea:.0f} J/mol")
print(f"R2 value: {r_squared:.2f}")

plt.figure(figsize=(7,5))
plt.scatter(inv_T, lnk, color='green', label="Original Data")
plt.plot(inv_T, lin_model.predict(inv_T.to_numpy().reshape(-1,1)), label="Linear Fit", color='orange')
plt.xlabel("1/T (1/K)")
plt.ylabel("ln(k)")
plt.title("Arrhenius Plot of Sintered sample")
plt.legend()
plt.grid()
plt.show()

# 4. Estimation of reaction order

## For the NO:

In [ ]:
# Reaction order of NO for fresh catalyst
NO_conc = data2.loc[[2,12,13],'[NO] (ppm)']
rate = data2.loc[[2,12,13],'Rate (µmol/g·s)']

# natural logarithm
ln_NO = np.log(NO_conc)
ln_rate = np.log(rate)
ln_NO_np = ln_NO.to_numpy()  # converting to numpy
ln_rate_np = ln_rate.to_numpy()

# Calculating rxn order using np.gradient
reaction_order_NO = np.gradient(ln_rate_np, ln_NO_np)

for i in range(len(NO_conc)):
    print(f"At NO = {NO_conc.iloc[i]} ppm, Reaction order ≈ {reaction_order_NO[i]:.2f} in fresh sample.")
    
# Reaction order of NO for sintered catalyst
NO_conc = data3.loc[[1,2,6,7],'[NO] (ppm)']
rate = data3.loc[[1,2,6,7],'Rate (µmol/g·s)']

# natural logarithm
ln_NO = np.log(NO_conc)
ln_rate = np.log(rate)
ln_NO_np = ln_NO.to_numpy()  # converting to numpy
ln_rate_np = ln_rate.to_numpy()

# Calculating rxn order using np.gradient
reaction_order_NO = np.gradient(ln_rate_np, ln_NO_np)

for i in range(len(NO_conc)):
    print(f"At NO = {NO_conc.iloc[i]} ppm, Reaction order ≈ {reaction_order_NO[i]:.2f} in sintered sample.")

## For O2:

In [ ]:
# Reaction order of O2 for fresh catalyst
O2_conc = data2.loc[[11,14,15],'[O₂] (%)']
rate = data2.loc[[11,14,15],'Rate (µmol/g·s)']

# natural logarithm
ln_O2 = np.log(O2_conc)
ln_rate = np.log(rate)
ln_O2_np = ln_O2.to_numpy()  # converting to numpy
ln_rate_np = ln_rate.to_numpy()

# Calculating rxn order using np.gradient
reaction_order_O2 = np.gradient(ln_rate_np, ln_O2_np)

for i in range(len(O2_conc)):
    print(f"At O2 = {O2_conc.iloc[i]}%, Reaction order ≈ {reaction_order_O2[i]:.2f} in fresh sample.")
    
# Reaction order of O2 for sintered catalyst
O2_conc = data3.loc[[1,8,9],'[O₂] (%)']
rate = data3.loc[[1,8,9],'Rate (µmol/g·s)']

# natural logarithm
ln_O2 = np.log(O2_conc)
ln_rate = np.log(rate)
ln_O2_np = ln_O2.to_numpy()  # converting to numpy
ln_rate_np = ln_rate.to_numpy()

# Calculating rxn order using np.gradient
reaction_order_O2 = np.gradient(ln_rate_np, ln_O2_np)

for i in range(len(O2_conc)):
    print(f"At O2 = {O2_conc.iloc[i]}%, Reaction order ≈ {reaction_order_O2[i]:.2f} in sintered sample.")

## For NO2

In [ ]:
#fresh sample
NO2_conc = data2.loc[[16,17,18],'[NO₂] (ppm)']
rate = data2.loc[[16,17,18],'Rate (µmol/g·s)']

# natural logarithm
ln_NO2 = np.log(NO2_conc)
ln_rate = np.log(rate)
ln_NO2_np = ln_NO2.to_numpy()  # converting to numpy
ln_rate_np = ln_rate.to_numpy()

# Calculating rxn order using np.gradient
reaction_order_NO2 = np.gradient(ln_rate_np, ln_NO2_np)

for i in range(len(NO2_conc)):
    print(f"At NO2 = {NO2_conc.iloc[i]} ppm, Reaction order ≈ {reaction_order_NO2[i]:.2f} in fresh sample.")

#sintered sample
NO2_conc = data3.loc[[2,3],'[NO₂] (ppm)']
rate = data3.loc[[2,3],'Rate (µmol/g·s)']

# natural logarithm
ln_NO2 = np.log(NO2_conc)
ln_rate = np.log(rate)
ln_NO2_np = ln_NO2.to_numpy()  # converting to numpy
ln_rate_np = ln_rate.to_numpy()

# Calculating rxn order using np.gradient
reaction_order_NO2 = np.gradient(ln_rate_np, ln_NO2_np)

for i in range(len(NO2_conc)):
    print(f"At NO2 = {NO2_conc.iloc[i]} ppm, Reaction order ≈ {reaction_order_NO2[i]:.2f} in sintered sample.")